In [1]:
 # Import libraries
import pandas as pd
import pickle as pk

# import libraries for api
import os
from dotenv import load_dotenv 
import requests
import json

# import libraries for vizualization
import plotly.express as px
import panel as pn
from pprint import pprint
import folium 
pn.extension(sizing_mode="stretch_width")
from jupyter_bokeh.widgets   import BokehModel
import base64
from folium import IFrame

In [2]:
### Open data from a pickle : Zillow_data, which represent the houses currently for sale
filename = "Zillow_data"
infile = open(filename,'rb')
new_dict = pk.load(infile)
infile.close()

In [3]:
### transform to json
z_for_sale_resp =new_dict.json()

In [4]:
### transform json to datafram
current_zillow_df = pd.json_normalize(data=z_for_sale_resp["props"])
print('num of rows:', len(current_zillow_df))
print('num of columns:', len(current_zillow_df.columns))

current_zillow_df.head()
print(current_zillow_df.loc[0,"imgSrc"])

num of rows: 40
num of columns: 18
https://photos.zillowstatic.com/fp/c528e4507f6a3a9faa6a6397b3b76d10-p_e.jpg


In [5]:
### make a list of the zpid
zpid_list = current_zillow_df["zpid"].tolist()
current_zillow_df.columns

Index(['bathrooms', 'propertyType', 'lotAreaValue', 'address', 'imgSrc',
       'price', 'listingDateTime', 'listingStatus', 'zpid', 'longitude',
       'latitude', 'lotAreaUnit', 'daysOnZillow', 'bedrooms', 'country',
       'currency', 'livingArea', 'hasImage'],
      dtype='object')

In [6]:
### Select columns that we will need for the analysis
current_zillow_df.columns
current_price_df = current_zillow_df[[ "zpid","propertyType" , "price", "livingArea" , "bathrooms" , "bedrooms" ,"address", "listingStatus" , "latitude" , "longitude", "imgSrc"]].copy()
current_price_df.set_index("zpid", inplace=True)


#Pull only the zip code from the address column
current_price_df["address"] = current_price_df["address"].str[-5:]

# Change the column names to improve readability
current_price_df.rename(columns = {
        "livingArea" : "Area in sqft.",
        "propertyType" : "Type of Property",
        "address" : "zip code",
        "listingStatus" : "Status",
        "imgSrc" : "Image property"
        }, inplace = True)

current_price_df.head()

,Type of Property,price,Area in sqft.,bathrooms,bedrooms,zip code,Status,latitude,longitude,Image property
zpid,,,,,,,,,,
10428068,SINGLE_FAMILY,150000,1440,2,4,19120,FOR_SALE,40.026787,-75.130410,https://photos.zillowstatic.com/fp/c528e4507f6...
10291269,SINGLE_FAMILY,325000,1920,2,4,19124,FOR_SALE,40.024803,-75.091110,https://photos.zillowstatic.com/fp/84229141ec1...
2068504637,SINGLE_FAMILY,869000,3857,4,5,19144,FOR_SALE,40.037050,-75.181816,https://photos.zillowstatic.com/fp/b90b6b03745...
10486635,SINGLE_FAMILY,267000,4520,4,5,19131,FOR_SALE,39.996180,-75.227104,https://photos.zillowstatic.com/fp/458505836a3...
2071261604,SINGLE_FAMILY,144000,2007,3,5,19111,FOR_SALE,40.050960,-75.092926,https://photos.zillowstatic.com/fp/04cc332dd77...


In [7]:
### Open data from a pickle : Zillow_old_data, which represent the houses recently sold and similar to the current one for sale
filename = "Zillow_old_data"
infile = open(filename,'rb')
new_source = pk.load(infile)
infile.close()

In [8]:
### transform to json
z_recently_sold_resp =new_source.json()

In [9]:
### transform json to datafram
sold_zillow_df = pd.json_normalize(data=z_recently_sold_resp)
print('num of rows:', len(sold_zillow_df))
print('num of columns:', len(sold_zillow_df.columns))

sold_zillow_df.head(20)

num of rows: 5
num of columns: 18


,dateSold,bedrooms,homeStatus,latitude,miniCardPhotos,zpid,longitude,bathrooms,livingArea,livingAreaUnits,homeType,currency,lastSoldPrice,price,address.city,address.state,address.streetAddress,address.zipcode
0,1610582400000,5,RECENTLY_SOLD,40.055460,[{'url': 'https://photos.zillowstatic.com/fp/1...,10214276,-75.195090,3,2138,Square Feet,SINGLE_FAMILY,USD,470000,470000,Philadelphia,PA,7139 Lincoln Dr,19119
1,1619740800000,3,RECENTLY_SOLD,40.051693,[{'url': 'https://photos.zillowstatic.com/fp/f...,10284714,-75.194620,4,2200,Square Feet,SINGLE_FAMILY,USD,477500,477500,Philadelphia,PA,403 W Ellet St,19119
2,1614556800000,4,RECENTLY_SOLD,40.051390,[{'url': 'https://photos.zillowstatic.com/fp/6...,10284790,-75.195480,3,2400,Square Feet,SINGLE_FAMILY,USD,443000,443000,Philadelphia,PA,426 Glen Echo Rd,19119
3,1629158400000,5,RECENTLY_SOLD,40.057297,[{'url': 'https://photos.zillowstatic.com/fp/c...,103779525,-75.193504,3,2665,Square Feet,SINGLE_FAMILY,USD,480000,480000,Philadelphia,PA,7141 Cresheim Rd,19119
4,1634256000000,3,RECENTLY_SOLD,40.051323,[{'url': 'https://photos.zillowstatic.com/fp/4...,333709815,-75.196860,3,1760,Square Feet,SINGLE_FAMILY,USD,460000,460000,Philadelphia,PA,504 W Mount Pleasant Ave,19119


In [10]:
### Select columns that we will need for the analysis
sold_zillow_df.columns
sold_price_df = sold_zillow_df[[ "zpid", "homeType", "lastSoldPrice", "livingArea" , "bathrooms" , "bedrooms" , "address.zipcode",  "homeStatus" , "latitude" , "longitude", "miniCardPhotos"]].copy()
sold_price_df.set_index("zpid", inplace=True)

# Change the column names to improve readability
sold_price_df.rename(columns = {
        "homeType" : "Type of Property",
        "lastSoldPrice" : "Price",
        "livingArea" : "Area in sqft.",
        "address.zipcode" :"zip code",
        "homeStatus" : "Status",
        "miniCardPhotos" : "Image property"
        }, inplace = True)

sold_price_df.head(40)

,Type of Property,Price,Area in sqft.,bathrooms,bedrooms,zip code,Status,latitude,longitude,Image property
zpid,,,,,,,,,,
10214276,SINGLE_FAMILY,470000,2138,3,5,19119,RECENTLY_SOLD,40.055460,-75.195090,[{'url': 'https://photos.zillowstatic.com/fp/1...
10284714,SINGLE_FAMILY,477500,2200,4,3,19119,RECENTLY_SOLD,40.051693,-75.194620,[{'url': 'https://photos.zillowstatic.com/fp/f...
10284790,SINGLE_FAMILY,443000,2400,3,4,19119,RECENTLY_SOLD,40.051390,-75.195480,[{'url': 'https://photos.zillowstatic.com/fp/6...
103779525,SINGLE_FAMILY,480000,2665,3,5,19119,RECENTLY_SOLD,40.057297,-75.193504,[{'url': 'https://photos.zillowstatic.com/fp/c...
333709815,SINGLE_FAMILY,460000,1760,3,3,19119,RECENTLY_SOLD,40.051323,-75.196860,[{'url': 'https://photos.zillowstatic.com/fp/4...


In [11]:
total_price_df = pd.concat([current_price_df,sold_price_df ], axis=0, join="inner")

total_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 10428068 to 333709815
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Type of Property  45 non-null     object 
 1   Area in sqft.     45 non-null     int64  
 2   bathrooms         45 non-null     int64  
 3   bedrooms          45 non-null     int64  
 4   zip code          45 non-null     object 
 5   Status            45 non-null     object 
 6   latitude          45 non-null     float64
 7   longitude         45 non-null     float64
 8   Image property    45 non-null     object 
dtypes: float64(2), int64(3), object(4)
memory usage: 3.5+ KB


In [12]:
import branca

In [28]:
PhillyMap = folium.Map(location = [39.9526, -75.1652])


In [29]:
f = branca.element.Figure()

In [38]:
for i in range(0,len(current_zillow_df["latitude"])):
    folium.Marker(
        location=[current_zillow_df.iloc[i]["latitude"], current_zillow_df.iloc[i]["longitude"]],
        popup= current_zillow_df.iloc[i]["address"],
        icon=folium.Icon(color="green", icon='home')
    ).add_to(PhillyMap)
    
for i in range(0,len(sold_zillow_df["latitude"])):
    folium.Marker(
        location=[sold_zillow_df.iloc[i]["latitude"], sold_zillow_df.iloc[i]["longitude"]],
        popup= sold_zillow_df.iloc[i]["address.streetAddress"],
        icon=folium.Icon(color="red", icon='home')
    ).add_to(PhillyMap)

In [39]:
Philly_Map_Pane = pn.pane.plot.Folium(PhillyMap, height=400)
Philly_Map_Pane

Folium(Map, height=400)